In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import warnings

In [2]:
#tickers data for download
tickers = ['^GSPC', 'BTC-USD', 'EURUSD=X'] 

data = yf.download(tickers, start='2015-01-01', end='2024-12-31')
data = data.loc[:, ['Close', 'Volume']]
data.columns = ['{}_{}'.format(price_type, ticker) for price_type, ticker in data.columns]
data.tail()

C:\Users\juanc\AppData\Local\Temp\ipykernel_27144\420465959.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start='2015-01-01', end='2024-12-31')
[*********************100%***********************]  3 of 3 completed


,Close_BTC-USD,Close_EURUSD=X,Close_^GSPC,Volume_BTC-USD,Volume_EURUSD=X,Volume_^GSPC
Date,,,,,,
2024-12-26,95795.515625,1.039955,6037.589844,47054980873,0.0,2.904530e+09
2024-12-27,94164.859375,1.042318,5970.839844,52419934565,0.0,3.159610e+09
2024-12-28,95163.929688,NaN,NaN,24107436185,NaN,NaN
2024-12-29,93530.226562,NaN,NaN,29635885267,NaN,NaN
2024-12-30,92643.210938,1.042938,5906.939941,56188003691,0.0,3.433250e+09


In [3]:
# Fill NaNs in volume columns with 0.0
volume_cols = data.filter(like='Volume_').columns
data[volume_cols] = data[volume_cols].fillna(0.0)

# Find volume columns where all values are zero or NaN to drop them
useless_volume_cols = [
    col for col in volume_cols if data[col].isna().all() or (data[col] == 0.0).all()
]
data = data.drop(columns=useless_volume_cols)

#fills NaN with previous values (assumes price stays the same the days that the market is closed)
data = data.fillna(method='ffill')
#fills initial NaN with following values (only initial rows)
data = data.fillna(method='bfill')
data.tail()

C:\Users\juanc\AppData\Local\Temp\ipykernel_27144\602548488.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')
C:\Users\juanc\AppData\Local\Temp\ipykernel_27144\602548488.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='bfill')


,Close_BTC-USD,Close_EURUSD=X,Close_^GSPC,Volume_BTC-USD,Volume_^GSPC
Date,,,,,
2024-12-26,95795.515625,1.039955,6037.589844,47054980873,2.904530e+09
2024-12-27,94164.859375,1.042318,5970.839844,52419934565,3.159610e+09
2024-12-28,95163.929688,1.042318,5970.839844,24107436185,0.000000e+00
2024-12-29,93530.226562,1.042318,5970.839844,29635885267,0.000000e+00
2024-12-30,92643.210938,1.042938,5906.939941,56188003691,3.433250e+09


In [4]:
data.to_csv('../data/financial_data.csv')